# Assignment 5 - Problem 1

### Setup

Import libraries

In [16]:
import numpy as np

Define functions

In [17]:
def parseVector(line):
    return np.array([float(x) for x in line.split(' ')])


def closestPoint(p, centers):
    bestI = 0
    closest = float("+inf")
    
    for i in range(len(centers)):
        temp = np.sum((p - centers[i]) ** 2)
        
        if temp < closest:
                closest = temp
                bestI = i
                
    return bestI




Import Spark

In [18]:
import os
import findspark
os.environ["PYSPARK_PYTHON"] = "python3"
findspark.init("spark-2.4.2-bin-hadoop2.7/",)

Initialize Spark

In [19]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .master("local[*]")
        .appName("LectureExample")
        .getOrCreate()
)
sc = spark.sparkContext

### Import Files

#### Import Centroids

In [20]:
lines = sc.textFile('c1.txt')
centroids = lines.map(parseVector)

#### Import Data

In [21]:
lines = sc.textFile('data.txt')
data = lines.map(parseVector)

### K-Means Implementation

In [25]:
# def kMeans(dataIN, kIN, centroidsIN, numIterIN): 
numIter = 0

closest = data.map(lambda p: (closestPoint(p, centroids), (p,1)))
pointStats = closest.reduceByKey(lambda p1_c1, p2_c2: (p1_c1[0] + p2_c2[0], p1_c1[1] + p2_c2[1]))

Traceback (most recent call last):
  File "spark-2.4.2-bin-hadoop2.7/python/pyspark/serializers.py", line 590, in dumps
    return cloudpickle.dumps(obj, 2)
  File "spark-2.4.2-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 863, in dumps
    cp.dump(obj)
  File "spark-2.4.2-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 260, in dump
    return Pickler.dump(self, obj)
  File "/home/jamster/anaconda3/lib/python3.7/pickle.py", line 437, in dump
    self.save(obj)
  File "/home/jamster/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/home/jamster/anaconda3/lib/python3.7/pickle.py", line 786, in save_tuple
    save(element)
  File "/home/jamster/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "spark-2.4.2-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 406, in save_function
    self.save_function_tuple(obj)
  File "spark-2.4.2-bin-hadoop2.

PicklingError: Could not serialize object: Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.

In [22]:
while numIter <= 25:
    
    
    newPoints = pointStats.map(lambda s: (s[0], s[1][0] / s[1][1])).collect()

    for (iK, p) in newPoints:
        centroids[iK] = p
        
    numIter = numIter + 1

Traceback (most recent call last):
  File "spark-2.4.2-bin-hadoop2.7/python/pyspark/serializers.py", line 590, in dumps
    return cloudpickle.dumps(obj, 2)
  File "spark-2.4.2-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 863, in dumps
    cp.dump(obj)
  File "spark-2.4.2-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 260, in dump
    return Pickler.dump(self, obj)
  File "/home/jamster/anaconda3/lib/python3.7/pickle.py", line 437, in dump
    self.save(obj)
  File "/home/jamster/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/home/jamster/anaconda3/lib/python3.7/pickle.py", line 786, in save_tuple
    save(element)
  File "/home/jamster/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "spark-2.4.2-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 406, in save_function
    self.save_function_tuple(obj)
  File "spark-2.4.2-bin-hadoop2.

PicklingError: Could not serialize object: Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.

In [ ]:
test = kMeans(data, 10, centroids, 25)